In [12]:
import apache_beam as beam
import pandas as pd
import glob
import os

from datetime import datetime

from apache_beam.options import pipeline_options
from apache_beam.options.pipeline_options import GoogleCloudOptions
from apache_beam.runners import DataflowRunner

import google.auth
import apache_beam.runners.interactive.interactive_beam as ib
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner

In [13]:
count = 0
def print_info(df):
    global count
    count = count + 1
    return {'df_num': count, 'columns': list(df.columns), 'shape': df.shape}

In [35]:
# Create pipeline object
p = beam.Pipeline(InteractiveRunner())

In [36]:
# Add pipeline components
csv_details =   (
                    p 
                    | 'List csv files' >> beam.Create(glob.glob('./data/*.csv'))
                    | 'Read csv files' >> beam.Map((pd.read_csv))
                    | 'Print csv details' >> beam.Map(print_info)
                 )

#### Visualize Pipeline

In [37]:
ib.show_graph(p)

##### Make sure tar file in './data' folder is un-tarred. That will yield 4 csv files.

In [17]:
start_time = datetime.now()
p.run().wait_until_finish()
end_time = datetime.now()

/root/apache-beam-2.33.0/lib/python3.7/site-packages/apache_beam/runners/worker/sdk_worker.py:640: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  bundle_processor.process_bundle(instruction_id))
/root/apache-beam-2.33.0/lib/python3.7/site-packages/apache_beam/runners/worker/sdk_worker.py:640: DtypeWarning: Columns (11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  bundle_processor.process_bundle(instruction_id))
/root/apache-beam-2.33.0/lib/python3.7/site-packages/apache_beam/runners/worker/sdk_worker.py:640: DtypeWarning: Columns (9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50) have mixed types.Specify dtype option on import or set low_memory=False.
  bundle_processor.process_bundle(instruction_id))


In [18]:
ib.show(csv_details)

### Time taken when using pipeline

In [19]:
print(f"Time taken: {(end_time - start_time).total_seconds()} Seconds")

Time taken: 13.906486 Seconds


In [8]:
start_time = datetime.now()
for csv_file in glob.glob('./data/*.csv'):
    df = pd.read_csv(csv_file)
    print_info(df)
end_time = datetime.now()

/root/apache-beam-2.33.0/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3173: DtypeWarning: Columns (11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)



Dataframe: 5
columns:  ['Series_reference', 'Period', 'Data_value', 'STATUS', 'UNITS', 'MAGNTUDE', 'Subject', 'Group', 'Series_title_1', 'Series_title_2', 'Series_title_3', 'Series_title_4', 'Series_title_5']
Shape:  (1118488, 13)


Dataframe: 6
columns:  ['Series_reference', 'Period', 'Data_value', 'STATUS', 'UNITS', 'MAGNTUDE', 'Subject', 'Group', 'Series_title_1', 'Series_title_2', 'Series_title_3', 'Series_title_4', 'Series_title_5']
Shape:  (25468, 13)



/root/apache-beam-2.33.0/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3173: DtypeWarning: Columns (9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)



Dataframe: 7
columns:  ['STATUS', 'SER_NBR', 'Series_reference', 'Period', 'Data_value', 'UNITS', 'MAGNTUDE', 'Subject', 'Group', 'Age Group 3 brackets', 'Age Group', 'Age Group 6 brackets', 'Duration of unemployment', 'Employed and Unemployed Persons, Full-Time and Part-Time Status', 'Employment relationship', 'Employment status', 'Ethnic Single / Combination', 'Ethnic Total Response', 'Formal study status', 'Highest qualification', 'Hours Worked', 'Household Composition', 'Household Labour Force Status', 'Industry ANZSIC06', 'Industry ANZSIC06 Supplementary', 'Job', 'Job tenure', 'Labour force and education status', 'Labour Force Status', 'Main activity', 'Main job', 'Methods of seeking employment', 'Occupation ANZSCO Level 1', 'Percentage change from previous period and same period previous year', 'Persons Employed, Unemployed, Not in Labour Force (for current quarter)', 'Reason for leaving last job', 'Reason not seeking work', 'Reason not wanting work', 'Reasons not available for 

/root/apache-beam-2.33.0/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3173: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)



Dataframe: 8
columns:  ['Series_reference', 'Period', 'Data_value', 'STATUS', 'UNITS', 'MAGNTUDE', 'Subject', 'Group', 'Series_title_1', 'Series_title_2', 'Series_title_3', 'Series_title_4', 'Series_title_5']
Shape:  (174259, 13)



### Time taken when using python directly

In [9]:
print(f"Time taken: {(end_time - start_time).total_seconds()} Seconds")

Time taken: 12.626352 Seconds


##### Python looks slightly faster here. But when the same job is submitted to cloud dataflow, The process will run faster since it uses parallel computing in distributed systems and also it's scalable. 

## Run Pipeline as a dataflow job

In [38]:
# Setting up the Apache Beam pipeline options.
options = pipeline_options.PipelineOptions(flags=[])

# Sets the project to the default project in your current Google Cloud environment.
_, options.view_as(GoogleCloudOptions).project = google.auth.default()

# Sets the Google Cloud Region in which Cloud Dataflow runs.
options.view_as(GoogleCloudOptions).region = 'us-central1'

In [39]:
dataflow_gcs_location = 'gs://text-analysis-323506/dataflow_csv_reader'

In [40]:
# Dataflow Staging Location. This location is used to stage the Dataflow Pipeline and SDK binary.
options.view_as(GoogleCloudOptions).staging_location = '%s/staging' % dataflow_gcs_location

# Dataflow Temp Location. This location is used to store temporary files or intermediate results before finally outputting to the sink.
options.view_as(GoogleCloudOptions).temp_location = '%s/temp' % dataflow_gcs_location

In [41]:
# The directory to store the output files of the job.
output_gcs_location = '%s/output' % dataflow_gcs_location

In [42]:
# Specifying the Cloud Storage location to write `csvc_details` to,
# based on the `output_gcs_location` variable set earlier.
(csv_details | 'Write csv details to Cloud Storage' 
 >> beam.io.WriteToText(output_gcs_location + '/csv_details-output.txt'))

<PCollection[[42]: Write csv details to Cloud Storage/Write/WriteImpl/FinalizeWrite.None] at 0x7f2c2dd51510>

In [43]:
ib.show_graph(p)

In [27]:
# Submit job
# Important: Enable dataflow api if not enabled in google cloud platform console
pipeline_result = DataflowRunner().run_pipeline(p, options=options)